In [2]:
import numpy as np
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("..")
from utils import *
import warnings
warnings.filterwarnings("ignore")
import os
import psutil

pd.options.display.max_columns=1000
pd.options.display.max_rows=1000

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
from catboost import CatBoostClassifier, Pool

In [3]:
''' Data Load '''
train = pl.read_csv('/home/workspace/DACON/Click_predict/data/train.csv')
test = pl.read_csv('/home/workspace/DACON/Click_predict/data/test.csv')
train, test = to_pandas(train, test)
# train, test = load_data()
# train = sampling('/home/workspace/DACON/Click_predict/data/train.csv')
# test = pd.read_csv('/home/workspace/DACON/Click_predict/data/test.csv')

''' Memory Optimization '''
# train = reduce_mem_usage(train)
# test = reduce_mem_usage(test)
# print("k")

''' Preprocessing '''
train = preprocessing(train)
test = preprocessing(test, is_test = True)


---------------- Start MissingValue ----------------


100%|██████████| 26/26 [00:37<00:00,  1.42s/it]


---------------- Start Category ----------------


100%|██████████| 5/5 [04:39<00:00, 55.89s/it]


---------------- Change Dtype ----------------
Memory usage of dataframe is 8729.67 MB
Memory usage after optimization is: 6629.10 MB
Decreased by 24.1%
---------------- Start MissingValue ----------------


100%|██████████| 26/26 [00:05<00:00,  4.92it/s]


---------------- Start Category ----------------


100%|██████████| 5/5 [00:45<00:00,  9.15s/it]


---------------- Change Dtype ----------------
Memory usage of dataframe is 1350.43 MB
Memory usage after optimization is: 1038.79 MB
Decreased by 23.1%


In [4]:
''' Train / Vaild Split'''
X_train = train.drop(columns = ["Click"])
y_train = train["Click"]
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)
category_cols = list(X_train.select_dtypes(include=['category']).columns)

In [5]:
''' Modeling '''
train_pool = Pool(X_train, y_train ,cat_features = category_cols)
val_pool = Pool(X_valid, y_valid, cat_features = category_cols)

catb_params = {
    'iterations' : 400,
    'learning_rate': 0.05 ,
    "max_depth": 10,
    'eval_metric' :'AUC', #GPU 이용시 AUC말고 다른 매트릭으로 평가됨 
    'task_type' : 'GPU', #device는 굳이 전달 안 해도 None이 디폴트로, 자동으로 지정된다.
    'verbose': 1,
    'random_seed' : 721,
    'gpu_ram_part': 0.99, #gpu의 80%만 사용하라. default = 0.95 (95%)
    "used_ram_limit" :'80gb'
    }

In [6]:
catb = CatBoostClassifier(**catb_params)
catb.fit(train_pool, eval_set=val_pool, verbose=True)
catb.save_model('best_model3.cb')

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7237113	best: 0.7237113 (0)	total: 4.21s	remaining: 27m 58s
1:	total: 8.39s	remaining: 27m 48s
2:	total: 12.7s	remaining: 27m 56s
3:	total: 16.6s	remaining: 27m 25s
4:	total: 20.2s	remaining: 26m 36s
5:	test: 0.7433221	best: 0.7433221 (5)	total: 24.7s	remaining: 27m 1s
6:	total: 28.6s	remaining: 26m 47s
7:	total: 33.3s	remaining: 27m 9s
8:	total: 37.1s	remaining: 26m 53s
9:	total: 41.8s	remaining: 27m 11s
10:	test: 0.7468270	best: 0.7468270 (10)	total: 45.8s	remaining: 26m 58s
11:	total: 49.7s	remaining: 26m 48s
12:	total: 53.6s	remaining: 26m 35s
13:	total: 57.5s	remaining: 26m 25s
14:	total: 1m 1s	remaining: 26m 12s
15:	test: 0.7496129	best: 0.7496129 (15)	total: 1m 4s	remaining: 25m 50s
16:	total: 1m 8s	remaining: 25m 32s
17:	total: 1m 12s	remaining: 25m 28s
18:	total: 1m 15s	remaining: 25m 22s
19:	total: 1m 19s	remaining: 25m 17s
20:	test: 0.7522698	best: 0.7522698 (20)	total: 1m 24s	remaining: 25m 17s
21:	total: 1m 27s	remaining: 25m 3s
22:	total: 1m 31s	remaining: 24m 

: 

In [ ]:
# 전처리 전에 best: 0.7682916 (100)
# 전처리 후에 best: 0.7684399 (100)
y_pred = catb.predict_proba(X_valid)[:,1]
# ROC AUC 평가
score = roc_auc_score(y_valid, y_pred)
print(score)

0.7745402543496831


In [7]:
pred = catb.predict_proba(test)

In [8]:
sample_submission = pd.read_csv('/home/workspace/DACON/Click_predict/data/sample_submission.csv')
sample_submission['Click'] = pred
sample_submission.to_csv('click_ratio(catb2).csv',index=False)

OSError: Cannot save file into a non-existent directory: '/home/sgh/yes/envs/DACON'

In [34]:
# 시스템 메모리 정보 가져오기
mem = psutil.virtual_memory()

# 남은 램 용량 확인
available_memory = mem.available / (1024 ** 3)  # GB 단위로 변환
print("Available Memory:", available_memory, "GB")

Available Memory: 40.082908630371094 GB


# Models

## Catboost

In [47]:
from catboost import CatBoostClassifier, Pool

train_pool = Pool(X_train1, y_train1 ,cat_features = categorical_feat1)
val_pool = Pool(X_test1, y_test1, cat_features = categorical_feat1)


catb_params = {
    'iterations' : 200,
    'learning_rate': 0.05 ,
    "max_depth": 10,
    'eval_metric' :'AUC', #GPU 이용시 AUC말고 다른 매트릭으로 평가됨 
    'task_type' : 'GPU', #device는 굳이 전달 안 해도 None이 디폴트로, 자동으로 지정된다.
    'verbose': 1,
    'random_seed' : 721,
    'gpu_ram_part': 0.9 #gpu의 80%만 사용하라. default = 0.95 (95%)
    #'used_ram_limit': '20gb' #cpu가 20gb만 이용하도록 함
    }

catb = CatBoostClassifier(**catb_params)
catb.fit(train_pool, eval_set=val_pool, verbose=True)

# 예측 확률 계산
#y_pred = catb.predict_proba(X_test1)[:,1]

# ROC AUC 평가
#score = roc_auc_score(y_test1, y_pred)
#print(score)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7243258	best: 0.7243258 (0)	total: 2.91s	remaining: 9m 40s
1:	total: 6.15s	remaining: 10m 9s
2:	total: 9.14s	remaining: 10m
3:	total: 12.7s	remaining: 10m 24s
4:	total: 15.9s	remaining: 10m 21s
5:	test: 0.7363940	best: 0.7363940 (5)	total: 19.2s	remaining: 10m 21s
6:	total: 22.2s	remaining: 10m 12s
7:	total: 25.9s	remaining: 10m 22s
8:	total: 29.2s	remaining: 10m 20s
9:	total: 32.9s	remaining: 10m 24s
10:	test: 0.7396747	best: 0.7396747 (10)	total: 36.4s	remaining: 10m 25s
11:	total: 39.8s	remaining: 10m 23s
12:	total: 43.1s	remaining: 10m 19s
13:	total: 46.2s	remaining: 10m 14s
14:	total: 48.9s	remaining: 10m 3s
15:	test: 0.7426370	best: 0.7426370 (15)	total: 52.1s	remaining: 9m 58s
16:	total: 55.4s	remaining: 9m 56s
17:	total: 58.7s	remaining: 9m 53s
18:	total: 1m 1s	remaining: 9m 47s
19:	total: 1m 4s	remaining: 9m 41s
20:	test: 0.7459271	best: 0.7459271 (20)	total: 1m 8s	remaining: 9m 41s
21:	total: 1m 11s	remaining: 9m 35s
22:	total: 1m 14s	remaining: 9m 32s
23:	total: 1

In [24]:
import joblib

# 모델 저장 경로
model_path = "/home/sgh/yes/envs/DACON/models/"

# 경로가 존재하지 않으면 생성
os.makedirs(model_path, exist_ok=True)

# 모델 저장
catb.save_model(os.path.join(model_path, "click_ratio_catb(vanilla).pth"))

In [48]:
print(X_test.columns)
print(X_train1.columns)

Index(['F02', 'F04', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'F12', 'F13',
       'F14', 'F16', 'F17', 'F18', 'F19', 'F21', 'F22', 'F23', 'F24', 'F25',
       'F28', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38',
       'F39'],
      dtype='object')
Index(['F02', 'F04', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'F12', 'F13',
       'F14', 'F16', 'F17', 'F18', 'F19', 'F21', 'F22', 'F23', 'F24', 'F25',
       'F28', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38',
       'F39'],
      dtype='object')


In [43]:
#sample_submission = sample_submission.to_pandas()

y_submission_pred2 = catb.predict_proba(X_test)[:,1]

sample_submission['Click'] = y_submission_pred2
submission = sample_submission.copy()

submission.to_csv('/home/sgh/yes/envs/DACON/click_ratio(catb2).csv',index=False)